In [ ]:
def parse_graph_freq_webplotdigitizer(filename):
    """ """
    # from 20Hz to 20kHz, log(2)~0.3                                                                                                                                                 
    ref_freq = np.logspace(1+math.log10(2), 4+math.log10(2), 203)
    #                                                                                                                                                                                
    with open(filename, 'r') as f:
        # data are stored in a json file.                                                                                                                                            
        speaker_data = json.load(f)
        # store all results                                                                                                                                                          
        res = []
        for col in speaker_data['datasetColl']:
            data = col['data']
            # sort data
            udata = np.array([[data[d]['value'][0], data[d]['value'][1]] for d in range(0, len(data))])
            sdata = np.sort(udata.view('i8,i8'), order=['f1'], axis=0)
            print(col['name'], len(sdata))                                                                                                                                         
            print(sdata)                                                                                                                                                           
            # since sdata and freq_ref are both sorted, iterate over both                                                                                                            
            ref_p = 0
            for di in range(0, len(sdata)-1):
                d = sdata[di]
                dn = sdata[di+1]
                fr = d[0]
                db = d[1]
                frn = dn[0]
                dbn = dn[1]
                # remove possible errors                                                                                                                                             
                if fr == frn:
                    continue
                # look for closest match                                                                                                                                             
                while ref_freq[ref_p] <= fr:
                    if ref_p >= len(ref_freq)-1:
                        break
                    ref_p += 1
                # if ref_f is too large, skip                                                                                                                                        
                ref_f = ref_freq[ref_p]
                if ref_f > frn:
                    continue
                # linear interpolation                                                                                                                                               
                ref_db = db+((dbn-db)*(ref_f-fr))/(frn-fr)
                # print('fr={:.2f} fr_ref={:.2f} fr_n={:.2f} \                                                                                                                       
                #       db={:.1f} db_ref={:.1f} db_n={:.1f}'\                                                                                                                        
                #      .format(fr, ref_f, frn,                                                                                                                                       
                #              db, ref_db,          dbn))                                                                                                                            
                res.append([ref_f, ref_db, col['name']])

        # build dataframe                                                                                                                                                            
        ares = np.array(res)
        df = pd.DataFrame({'Freq': ares[:, 0], 'dB': ares[:, 1], 'Measurements': ares[:, 2]})
        # print(df)                                                                                                                                                                  
        return 'CEA2034', df

In [ ]:
import math
import numpy as np
import json
import pandas as pd
import altair as alt
from src.spinorama.graph import graph_spinorama,graph_params_default

In [ ]:
file = 'datas/Vendors/JBL/JBL 4367/JBL 4367.json'
dfs = parse_graph_freq_webplotdigitizer(file)
df = dfs[1]
df.dB

In [ ]:
graph_spinorama(df, graph_params_default)